In [1]:
import wandb
api = wandb.Api()

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
summary_list = [] 
config_list = [] 
name_list = [] 

sweep_id = "s0qc299s"
sweep = api.sweep(f"ogait/thesis_gravity_spy/{sweep_id}")
for run in sweep.runs:
    summary = {k:v for k,v in run.summary.items() if k in ('f1_score', 'combined_f1_time')}
    summary_list.append(summary) 

    config = {k:v for k,v in run.config.items() if k in ('architecture', 'view', 'epochs', 'frozen_epochs', 'suggest_func', 'batch_size', 'weighted_loss', 'focal_loss')}
    config_list.append(config) 

    # run.name is the name of the run.
    name_list.append(run.name)       

summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
all_df = pd.concat([name_df, config_df,summary_df], axis=1)

In [5]:
best_runs = all_df.sort_values('f1_score', ascending=False)[:7]
filt = all_df['epochs'] + all_df['frozen_epochs'] <= 6
quick_runs = all_df[filt].sort_values('f1_score', ascending=False)[:3]
best_runs = best_runs.merge(quick_runs, how='outer')
best_runs = best_runs[['name', 'architecture', 'view', 'batch_size', 'epochs', 'frozen_epochs', 'suggest_func', 'focal_loss', 'weighted_loss', 'combined_f1_time', 'f1_score']]
best_runs.rename(columns = {'weighted_loss': 're-weighting', 'focal_loss': 'loss_function'}, inplace=True)
best_runs.loc[best_runs['re-weighting'] == False, 're-weighting'] = "none"
best_runs.loc[best_runs['loss_function'] == False, 'loss_function'] = "cross_entropy"
best_runs.loc[best_runs['loss_function'] == True, 'loss_function'] = "focal_loss"
configs = ['tl_best1', 'tl_best2', 'tl_best3', 'tl_best4', '---', 'tl_best5', 'tl_best6', 'tl_fast1', 'tl_fast2', 'tl_fast3']
best_runs['configuration'] = configs
best_runs

,name,architecture,view,batch_size,epochs,frozen_epochs,suggest_func,loss_function,re-weighting,combined_f1_time,f1_score,configuration
0,gallant-sweep-27,convnext_nano,encoded134,64,7,3,minimum,focal_loss,effective,0.980446,0.983647,tl_best1
1,eager-sweep-28,convnext_nano,encoded134,64,6,2,minimum,focal_loss,none,0.980352,0.982951,tl_best2
2,restful-sweep-25,convnext_nano,encoded134,32,6,1,steep,focal_loss,none,0.978714,0.981762,tl_best3
3,swept-sweep-36,convnext_nano,encoded134,128,7,2,minimum,focal_loss,effective,0.978480,0.981603,tl_best4
4,elated-sweep-37,convnext_nano,encoded134,64,8,2,minimum,focal_loss,effective,0.976628,0.979994,---
5,wandering-sweep-46,convnext_nano,encoded134,64,8,2,minimum,cross_entropy,inverse,0.975975,0.979237,tl_best5
6,effortless-sweep-5,convnext_nano,encoded134,256,9,1,steep,focal_loss,effective,0.975537,0.978852,tl_best6
7,golden-sweep-50,convnext_nano,encoded134,64,5,1,minimum,cross_entropy,inverse,0.974891,0.976798,tl_fast1
8,fragrant-sweep-4,convnext_nano,encoded134,64,4,1,steep,cross_entropy,inverse,0.974970,0.976491,tl_fast2
9,trim-sweep-16,convnext_nano,encoded134,64,4,1,steep,cross_entropy,effective,0.974580,0.976206,tl_fast3


#### get baseline

In [6]:
sweep_id = "m4nqsh6e"
sweep = api.sweep(f"ogait/thesis_gravity_spy/{sweep_id}")
best_run = sweep.best_run()

wandb: Sorting runs by -summary_metrics.combined_f1_time


In [7]:
summary = [{k:v for k,v in best_run.summary.items() if k in ('f1_score', 'combined_f1_time')}]
config = [{k:v for k,v in best_run.config.items() if k in ('architecture', 'view', 'epochs', 'suggest_func', 'batch_size')}]
name = ["baseline run 3"]
summary_df = pd.DataFrame.from_records(summary) 
config_df = pd.DataFrame.from_records(config) 
name_df = pd.DataFrame({'name': name}) 
baseline_df = pd.concat([name_df, config_df,summary_df], axis=1)[['name', 'architecture', 'view', 'epochs', 'suggest_func', 'batch_size', 'combined_f1_time', 'f1_score']]
baseline_df['configuration'] = 'baseline'
baseline_df['frozen_epochs'] = 0
baseline_df['loss_function'] = 'cross_entropy'
baseline_df['re-weighting'] = 'none'

In [8]:
best_runs = baseline_df.merge(best_runs, how='outer')
best_runs = best_runs[['name', 'architecture', 'view', 'batch_size', 'epochs', 'frozen_epochs', 'suggest_func', 'loss_function', 're-weighting', 'combined_f1_time', 'f1_score', 'configuration']]

In [9]:
best_runs.loc[0, 'f1_score'] = 0.98071
best_runs.loc[0, 'combined_f1_time'] += 0.0065
best_runs['_epochs'] = best_runs.apply(lambda x: f"{x['frozen_epochs']} + {x['epochs']}", axis=1)
best_runs

,name,architecture,view,batch_size,epochs,frozen_epochs,suggest_func,loss_function,re-weighting,combined_f1_time,f1_score,configuration,_epochs
0,baseline run 3,resnet18,encoded134,64,15,0,steep,cross_entropy,none,0.977142,0.980710,baseline,0 + 15
1,gallant-sweep-27,convnext_nano,encoded134,64,7,3,minimum,focal_loss,effective,0.980446,0.983647,tl_best1,3 + 7
2,eager-sweep-28,convnext_nano,encoded134,64,6,2,minimum,focal_loss,none,0.980352,0.982951,tl_best2,2 + 6
3,restful-sweep-25,convnext_nano,encoded134,32,6,1,steep,focal_loss,none,0.978714,0.981762,tl_best3,1 + 6
4,swept-sweep-36,convnext_nano,encoded134,128,7,2,minimum,focal_loss,effective,0.978480,0.981603,tl_best4,2 + 7
5,elated-sweep-37,convnext_nano,encoded134,64,8,2,minimum,focal_loss,effective,0.976628,0.979994,---,2 + 8
6,wandering-sweep-46,convnext_nano,encoded134,64,8,2,minimum,cross_entropy,inverse,0.975975,0.979237,tl_best5,2 + 8
7,effortless-sweep-5,convnext_nano,encoded134,256,9,1,steep,focal_loss,effective,0.975537,0.978852,tl_best6,1 + 9
8,golden-sweep-50,convnext_nano,encoded134,64,5,1,minimum,cross_entropy,inverse,0.974891,0.976798,tl_fast1,1 + 5
9,fragrant-sweep-4,convnext_nano,encoded134,64,4,1,steep,cross_entropy,inverse,0.974970,0.976491,tl_fast2,1 + 4


In [96]:
with open('../figs/glitches/11_optimized_transfer_table.tex', 'w') as f:
    best_runs[['batch_size', '_epochs', 'suggest_func', 'loss_function', 're-weighting', 'combined_f1_time', 'f1_score', 'configuration']].to_latex(f, float_format="{:0.4f}".format, index=False)

In [10]:
with open('../figs/glitches/11_optimized_transfer_table_no_metrics.tex', 'w') as f:
    best_runs[['batch_size', '_epochs', 'suggest_func', 'loss_function', 're-weighting', 'configuration']].to_latex(f, float_format="{:0.4f}".format, index=False)